In [1]:
import random
from tokenize import Ignore, String
from faker import Faker
import pandas as pd
import datetime as dt
from datetime import date
import sys
import numpy as np
import os
from pyparsing import nums
from torch import real
import random_address

class TestDataGen(BaseOEAModule):
    def __init__(self, source_folder='test_data'):
        BaseOEAModule.__init__(self, oea, source_folder)

        self.faker = Faker('en_US')
        self.edlevel = 'high'
        #opt prek, elementary, mid, high, hed

        students = {
        'Gender': [],
        'FirstName': [],
        'MiddleName': [],
        'LastName': [],
        'StudentID': [],
        'Birthday': [],
        'School': [],
        'SchoolID': [],
        'Grade': [],
        'Performance': [],
        'HispanicLatino': [],
        'Race': [],
        'Flag': [],
        'Email': [],
        'Phone': [],
        'Address': [],
        'City': [],
        'State': [],
        'Zipcode': []
        }
        self.students = pd.DataFrame(students, dtype=object)

        schools = {
            'SchoolName':[],
            'SchoolID':[]
        }
        self.schools = pd.DataFrame(schools, dtype=object)

    def gen_base_tables(self, numstudents, numschools):
        self._gen_students(numstudents)
        self._gen_schools(numschools)
        self._assign_schools()
        dfStudents = spark.createDataFrame(self.students)
        dfSchools = spark.createDataFrame(self.schools)
        oea.land('test_data/gen_base_truth_tables', 'students', dfStudents)
        oea.land('test_data/gen_base_truth_tables', 'schools', dfSchools)

    # helper methods

    def __get_age(self, born):
        today = date.today()
        return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

    def __gen_student(self):
        gender = random.choices(['M','F','O'],weights=[0.45,0.5,0.05])
        if gender == ['M']:
            firstname = self.faker.first_name_male()
            middlename = self.faker.first_name_male()
        elif gender == ['F']: 
            firstname = self.faker.first_name_female()
            middlename = self.faker.first_name_female()
        elif gender == ['O']:
            firstname = self.faker.first_name_nonbinary()
            middlename = self.faker.first_name_nonbinary()
        lastname = self.faker.last_name()
        studentid = self.faker.uuid4()
        if self.edlevel == 'high':
            birthday = self.faker.date_of_birth(minimum_age=15,maximum_age=18)
        elif self.edlevel == 'middle':
            birthday = self.faker.date_of_birth(minimum_age=12,maximum_age=14)
        elif self.edlevel == 'elementary':
            birthday = self.faker.date_of_birth(minimum_age=6,maximum_age=11)
        elif self.edlevel == 'prek':
            birthday = self.faker.date_of_birth(minimum_age=2,maximum_age=5)
        # education
        school = ''
        schoolid = ''
        grade = self.__get_age(birthday) - 6
        performance = random.choices(['high','avg','low'], weights=[0.3,0.6,0.1])
        # demographics
        hispaniclatino = random.choices(['True','False'], weights=[0.189,0.811])
        race = random.choices(['white','blackafricanamerican','americanindianalaskanative','asian','nativehawaiianpacificislander','twoormoreraces'], weights=[0.758,0.136,0.013,0.061,0.003,0.029])
        flag = ''
        # contact
        email = f'{firstname}{lastname}@contoso.edu'
        phone = self.faker.phone_number()
        address = random_address.real_random_address_by_state('CA')
        city = address['city']
        state = address['state']
        zipcode = address['postalCode']
        address = address['address1']
        self.students.loc[len(self.students.index)] = [gender[0], firstname, middlename, lastname, studentid, birthday, school, \
        schoolid, grade, performance[0], hispaniclatino[0], race[0], flag, email, phone, address, city, state, zipcode]

    def _gen_students(self, numstudents=10):
        while numstudents > 0:
            self.__gen_student()
            numstudents = numstudents - 1

    def __gen_school(self):
        if self.edlevel == 'high':
            schoolname = f'{self.faker.last_name()} High'
        schoolid = self.faker.uuid4()
        self.schools.loc[len(self.schools.index)] = [schoolname, schoolid]

    def _gen_schools(self, numschools=3):
        while numschools > 0:
            self.__gen_school()
            numschools = numschools - 1

    def _assign_schools(self):
        i = len(self.students.index) - 1
        while i >= 0:
            school = random.randint(0, len(self.schools.index) - 1)
            self.students.at[i, 'School'] = self.schools.at[school, 'SchoolName']
            self.students.at[i, 'SchoolID'] = self.schools.at[school, 'SchoolID']
            i = i - 1